# 📊 Analyse Budgétaire CUCLM 2020 - Version Optimisée

Ce notebook analyse les données budgétaires de la Communauté Urbaine Caen la Mer (CUCLM) pour l'année 2020 en se concentrant sur :

- **Extraction des vraies communes** présentes dans le budget
- **Analyse par fonctions budgétaires** (Services généraux, Enseignement, Culture, etc.)
- **Calculs de dépenses/recettes/delta** par commune
- **Génération de données** pour le dashboard interactif

**Source :** `opendata-cuclm-budget-2020.csv` - Données officielles CUCLM

---

In [16]:
# 📦 Import des librairies nécessaires
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuration des graphiques
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_palette("husl")

print("✅ Librairies chargées avec succès")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📦 Numpy version: {np.__version__}")

✅ Librairies chargées avec succès
📦 Pandas version: 2.2.3
📦 Numpy version: 2.2.4


In [17]:
# 📂 Chargement et exploration des données budgétaires
print("🔄 Chargement du fichier CSV...")

# Chargement du fichier CSV (à adapter selon l'emplacement)
csv_path = 'pre-data/opendata-cuclm-budget-2020.csv'
try:
    df = pd.read_csv(csv_path, encoding='utf-8', sep=';')
    print(f"✅ Fichier chargé avec succès")
except FileNotFoundError:
    print("❌ Fichier non trouvé, tentative avec un autre chemin...")
    csv_path = 'opendata-cuclm-budget-2020.csv'
    df = pd.read_csv(csv_path, encoding='utf-8', sep=';')

print(f"📊 Dimensions du dataset: {df.shape}")
print(f"📊 Nombre de lignes: {df.shape[0]:,}")
print(f"📊 Nombre de colonnes: {df.shape[1]}")

print("\n📋 Colonnes disponibles:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")

print("\n🔍 Aperçu des premières lignes:")
df.head(3)

🔄 Chargement du fichier CSV...
✅ Fichier chargé avec succès
📊 Dimensions du dataset: (2814, 22)
📊 Nombre de lignes: 2,814
📊 Nombre de colonnes: 22

📋 Colonnes disponibles:
   1. COLLECTIVITE_CODE
   2. COLLECTIVITE_LIBELLE
   3. EXERCICE
   4. BUDGET_CODE
   5. BUDGET_LIBELLE
   6. CHAPITRE_CODE
   7. CHAPITRE_LIBELLE
   8. NATURE_CODE
   9. NATURE_LIBELLE
  10. FONCTION
  11. SOUS_FONCTION
  12. RUBRIQUE
  13. SECTION_CODE
  14. SECTION_LIBELLE
  15. TYPE_MOUVEMENT_CODE
  16. TYPE_MOUVEMENT_LIBELLE
  17. MONTANT_DEPLACE_DEPENSE
  18. MONTANT_DEPLACE_RECETTE
  19. MONTANT_BP
  20. MONTANT_BS
  21. MONTANT_DM
  22. MONTANT_RP

🔍 Aperçu des premières lignes:


,COLLECTIVITE_CODE,COLLECTIVITE_LIBELLE,EXERCICE,BUDGET_CODE,BUDGET_LIBELLE,CHAPITRE_CODE,CHAPITRE_LIBELLE,NATURE_CODE,NATURE_LIBELLE,FONCTION,...,SECTION_CODE,SECTION_LIBELLE,TYPE_MOUVEMENT_CODE,TYPE_MOUVEMENT_LIBELLE,MONTANT_DEPLACE_DEPENSE,MONTANT_DEPLACE_RECETTE,MONTANT_BP,MONTANT_BS,MONTANT_DM,MONTANT_RP
0,CUCLM,Communaute Urbaine Caen La Mer,2020.0,1.0,Budget principal,1.0,SOLDE D'EXECUTION DE LA SECTION D'INVESTISSEME...,1.0,SOLDE D'EXECUTION DE LA SECTION D'INVESTISSEME...,0.0,...,I,INVESTISSEMENT,D,DEPENSE,"0,00","0,00","0,00","28 195 412,27","0,00","0,00"
1,CUCLM,Communaute Urbaine Caen La Mer,2020.0,1.0,Budget principal,1.0,SOLDE D'EXECUTION DE LA SECTION D'INVESTISSEME...,1.0,SOLDE D'EXECUTION DE LA SECTION D'INVESTISSEME...,0.0,...,I,INVESTISSEMENT,R,RECETTE,"0,00","0,00","0,00","0,00","0,00","0,00"
2,CUCLM,Communaute Urbaine Caen La Mer,2020.0,1.0,Budget principal,2.0,RESULTAT DE FONCTIONNEMENT REPORTE,2.0,RESULTAT DE FONCTIONNEMENT REPORTE,0.0,...,F,FONCTIONNEMENT,D,DEPENSE,"0,00","0,00","0,00","0,00","0,00","0,00"


In [18]:
# 🔍 Analyse détaillée de la structure des données
print("🔍 ANALYSE DÉTAILLÉE DES DONNÉES")
print("=" * 50)

# Analyser les colonnes clés
print("📊 COLLECTIVITE_LIBELLE:")
print(df['COLLECTIVITE_LIBELLE'].unique())

print(f"\n📊 EXERCICE:")
print(df['EXERCICE'].unique())

print(f"\n📊 TYPE_MOUVEMENT_LIBELLE (recettes/dépenses):")
print(df['TYPE_MOUVEMENT_LIBELLE'].value_counts())

print(f"\n📊 SECTION_LIBELLE:")
print(df['SECTION_LIBELLE'].value_counts())

print(f"\n📊 Échantillon de CHAPITRE_LIBELLE (entités budgétaires):")
chapitres_sample = df['CHAPITRE_LIBELLE'].dropna().unique()[:20]
for i, chapitre in enumerate(chapitres_sample, 1):
    print(f"  {i:2d}. {chapitre}")

print(f"\n📊 FONCTION (codes de fonctions budgétaires):")
fonctions = df['FONCTION'].dropna().unique()
print(f"Fonctions trouvées: {sorted(fonctions)}")

# Analyser les montants
print(f"\n💰 ANALYSE DES MONTANTS:")
montant_cols = ['MONTANT_BP', 'MONTANT_BS', 'MONTANT_DM', 'MONTANT_RP']
for col in montant_cols:
    non_zero = df[df[col] != '0,00'][col].count()
    print(f"  • {col}: {non_zero:,} valeurs non nulles")

# Rechercher les communes dans CHAPITRE_LIBELLE
print(f"\n🏘️ RECHERCHE DE COMMUNES DANS CHAPITRE_LIBELLE:")
communes_cuclm = ['Caen', 'Ouistreham', 'Lion-sur-Mer', 'Louvigny', 'Ifs', 'Mondeville', 
                  'Hermanville', 'Benouville', 'Colleville', 'Mathieu']

for commune in communes_cuclm:
    mask = df['CHAPITRE_LIBELLE'].str.contains(commune, case=False, na=False)
    count = mask.sum()
    if count > 0:
        print(f"  ✅ {commune}: {count} entrées")
        # Montrer un exemple
        exemple = df[mask]['CHAPITRE_LIBELLE'].iloc[0]
        print(f"     Exemple: {exemple}")
    else:
        print(f"  ❌ {commune}: Aucune entrée")

print(f"\n📈 TOTAL DES LIGNES ANALYSÉES: {len(df):,}")

🔍 ANALYSE DÉTAILLÉE DES DONNÉES
📊 COLLECTIVITE_LIBELLE:
['Communaute Urbaine Caen La Mer' nan]

📊 EXERCICE:
[2020.   nan]

📊 TYPE_MOUVEMENT_LIBELLE (recettes/dépenses):
TYPE_MOUVEMENT_LIBELLE
DEPENSE    2245
RECETTE     568
Name: count, dtype: int64

📊 SECTION_LIBELLE:
SECTION_LIBELLE
FONCTIONNEMENT    1606
INVESTISSEMENT    1207
Name: count, dtype: int64

📊 Échantillon de CHAPITRE_LIBELLE (entités budgétaires):
   1. SOLDE D'EXECUTION DE LA SECTION D'INVESTISSEMENT REPORTE
   2. RESULTAT DE FONCTIONNEMENT REPORTE
   3. CHARGES A CARACTERE GENERAL
   4. CHARGES DE PERSONNEL ET FRAIS ASSIMILES
   5. ATTENUATIONS DE CHARGES
   6. ATTENUATIONS DE PRODUITS
   7. DEPENSES IMPREVUES
   8. VIREMENT DE LA SECTION DE FONCTIONNEMENT
   9. VIREMENT A LA SECTION D'INVESTISSEMENT
  10. PRODUITS DES CESSIONS D'IMMOBILISATIONS
  11. OPERATIONS D'ORDRE DE TRANSFERTS ENTRE SECTIONS
  12. OPERATIONS PATRIMONIALES
  13. DOTATIONS, FONDS DIVERS ET RESERVES
  14. Authie
  15. Benouville
  16. Bieville-Beuv

In [19]:
# 💰 Conversion et analyse des montants budgétaires
print("💰 CONVERSION DES MONTANTS")
print("=" * 40)

def nettoyer_montant(montant_str):
    """Convertit les montants du format CSV français en float"""
    if pd.isna(montant_str) or montant_str == '':
        return 0.0
    try:
        # Remplacer virgule par point et supprimer espaces
        montant_clean = str(montant_str).replace(',', '.').replace(' ', '').replace('\xa0', '')
        return float(montant_clean)
    except:
        return 0.0

# Convertir toutes les colonnes de montant
montant_cols = ['MONTANT_DEPLACE_DEPENSE', 'MONTANT_DEPLACE_RECETTE', 
                'MONTANT_BP', 'MONTANT_BS', 'MONTANT_DM', 'MONTANT_RP']

print("🔄 Conversion des colonnes de montants...")
for col in montant_cols:
    df[col + '_NUM'] = df[col].apply(nettoyer_montant)
    total = df[col + '_NUM'].sum()
    non_zero = (df[col + '_NUM'] != 0).sum()
    print(f"  • {col}: {total:,.0f}€ total, {non_zero:,} valeurs non-nulles")

# Calculer un montant total par ligne
df['MONTANT_TOTAL'] = (df['MONTANT_BP_NUM'] + df['MONTANT_BS_NUM'] + 
                       df['MONTANT_DM_NUM'] + df['MONTANT_RP_NUM'])

print(f"\n📊 MONTANT TOTAL GÉNÉRAL: {df['MONTANT_TOTAL'].sum():,.0f}€")

# Analyser par type de mouvement
print(f"\n📊 RÉPARTITION PAR TYPE DE MOUVEMENT:")
mouvement_stats = df.groupby('TYPE_MOUVEMENT_LIBELLE')['MONTANT_TOTAL'].agg(['sum', 'count'])
for mouvement, stats in mouvement_stats.iterrows():
    print(f"  • {mouvement}: {stats['sum']:,.0f}€ ({stats['count']:,} lignes)")

# Analyser les fonctions avec montants
print(f"\n🏛️ FONCTIONS BUDGÉTAIRES AVEC MONTANTS:")
fonction_stats = df[df['MONTANT_TOTAL'] > 0].groupby('FONCTION')['MONTANT_TOTAL'].agg(['sum', 'count']).sort_values('sum', ascending=False)
for fonction, stats in fonction_stats.head(10).iterrows():
    if pd.notna(fonction):
        print(f"  • Fonction {fonction}: {stats['sum']:,.0f}€ ({stats['count']:,} lignes)")

print("✅ Conversion des montants terminée")

💰 CONVERSION DES MONTANTS
🔄 Conversion des colonnes de montants...
  • MONTANT_DEPLACE_DEPENSE: 0€ total, 656 valeurs non-nulles
  • MONTANT_DEPLACE_RECETTE: 0€ total, 65 valeurs non-nulles
  • MONTANT_BP: 677,803,220€ total, 1,349 valeurs non-nulles
  • MONTANT_BS: 70,825,530€ total, 80 valeurs non-nulles
  • MONTANT_DM: 2,181,266€ total, 219 valeurs non-nulles
  • MONTANT_RP: 82,348,936€ total, 390 valeurs non-nulles

📊 MONTANT TOTAL GÉNÉRAL: 833,158,951€

📊 RÉPARTITION PAR TYPE DE MOUVEMENT:
  • DEPENSE: 416,579,476€ (2,245.0 lignes)
  • RECETTE: 416,579,476€ (568.0 lignes)

🏛️ FONCTIONS BUDGÉTAIRES AVEC MONTANTS:
  • Fonction 0.0: 527,513,394€ (348.0 lignes)
  • Fonction 8.0: 87,022,119€ (235.0 lignes)
  • Fonction 7.0: 80,054,496€ (228.0 lignes)
  • Fonction 6.0: 49,452,374€ (156.0 lignes)
  • Fonction 3.0: 43,563,768€ (290.0 lignes)
  • Fonction 5.0: 27,531,518€ (252.0 lignes)
  • Fonction 1.0: 14,747,321€ (31.0 lignes)
  • Fonction 2.0: 2,741,170€ (10.0 lignes)
  • Fonction 4.0:

In [20]:
# 🏘️ Extraction des vraies communes CUCLM
print("🏘️ EXTRACTION DES COMMUNES CUCLM")
print("=" * 50)

# Liste des communes CUCLM avec informations
communes_cuclm = {
    'Caen': {'lat': 49.1829, 'lon': -0.3707, 'population': 105512},
    'Ouistreham': {'lat': 49.2742, 'lon': -0.2581, 'population': 9451},
    'Lion-sur-Mer': {'lat': 49.3001, 'lon': -0.3189, 'population': 2503},
    'Louvigny': {'lat': 49.1656, 'lon': -0.4042, 'population': 2850},
    'Ifs': {'lat': 49.1433, 'lon': -0.3522, 'population': 11881},
    'Mondeville': {'lat': 49.1664, 'lon': -0.3139, 'population': 9312},
    'Hermanville-sur-Mer': {'lat': 49.2869, 'lon': -0.3075, 'population': 3021},
    'Benouville': {'lat': 49.2394, 'lon': -0.2881, 'population': 2069},
    'Colleville-Montgomery': {'lat': 49.2844, 'lon': -0.2892, 'population': 2507},
    'Mathieu': {'lat': 49.2503, 'lon': -0.3981, 'population': 2387}
}

def analyser_commune(nom_commune, df_data):
    """Analyse le budget d'une commune spécifique"""
    # Chercher toutes les variations du nom
    variations = [nom_commune, nom_commune.upper(), nom_commune.lower()]
    if '-' in nom_commune:
        variations.extend([nom_commune.replace('-', ' '), nom_commune.replace('-', '_')])
    
    # Créer le masque de recherche
    mask = pd.Series([False] * len(df_data))
    for variation in variations:
        mask |= df_data['CHAPITRE_LIBELLE'].str.contains(variation, case=False, na=False)
    
    if not mask.any():
        return None
    
    # Extraire les données de la commune
    commune_data = df_data[mask].copy()
    
    # Calculer les totaux par type de mouvement
    recettes = commune_data[commune_data['TYPE_MOUVEMENT_LIBELLE'] == 'RECETTE']['MONTANT_TOTAL'].sum()
    depenses = commune_data[commune_data['TYPE_MOUVEMENT_LIBELLE'] == 'DEPENSE']['MONTANT_TOTAL'].sum()
    total = recettes + depenses
    
    # Analyser par fonction
    fonctions_detail = {}
    for fonction in commune_data['FONCTION'].dropna().unique():
        fonction_data = commune_data[commune_data['FONCTION'] == fonction]
        rec_f = fonction_data[fonction_data['TYPE_MOUVEMENT_LIBELLE'] == 'RECETTE']['MONTANT_TOTAL'].sum()
        dep_f = fonction_data[fonction_data['TYPE_MOUVEMENT_LIBELLE'] == 'DEPENSE']['MONTANT_TOTAL'].sum()
        
        if rec_f > 0 or dep_f > 0:
            fonctions_detail[int(fonction)] = {
                'recettes': rec_f,
                'depenses': dep_f,
                'total': rec_f + dep_f
            }
    
    return {
        'nom': nom_commune,
        'nb_lignes': len(commune_data),
        'budget_total': total,
        'recettes': recettes,
        'depenses': depenses,
        'delta': recettes - depenses,
        'fonctions': fonctions_detail,
        'exemples_chapitre': commune_data['CHAPITRE_LIBELLE'].unique()[:3].tolist()
    }

# Analyser toutes les communes
communes_analysees = {}
print("🔍 Analyse des communes CUCLM...")

for nom_commune, info_commune in communes_cuclm.items():
    analyse = analyser_commune(nom_commune, df)
    
    if analyse:
        communes_analysees[nom_commune] = analyse
        print(f"\n✅ {nom_commune}:")
        print(f"  📊 {analyse['nb_lignes']} lignes budgétaires")
        print(f"  💰 Budget total: {analyse['budget_total']:,.0f}€")
        print(f"  📈 Recettes: {analyse['recettes']:,.0f}€")
        print(f"  📉 Dépenses: {analyse['depenses']:,.0f}€")
        print(f"  ⚖️ Delta: {analyse['delta']:,.0f}€")
        print(f"  🏛️ Fonctions: {len(analyse['fonctions'])}")
        print(f"  📋 Exemples: {', '.join(analyse['exemples_chapitre'])}")
    else:
        print(f"\n❌ {nom_commune}: Aucune donnée trouvée")

print(f"\n📊 RÉSUMÉ:")
print(f"  • Communes trouvées: {len(communes_analysees)}")
if communes_analysees:
    budget_total_communes = sum(c['budget_total'] for c in communes_analysees.values())
    print(f"  • Budget total communes: {budget_total_communes:,.0f}€")
    print(f"  • % du budget CUCLM: {budget_total_communes/df['MONTANT_TOTAL'].sum()*100:.1f}%")

🏘️ EXTRACTION DES COMMUNES CUCLM
🔍 Analyse des communes CUCLM...

✅ Caen:
  📊 36 lignes budgétaires
  💰 Budget total: 9,965,392€
  📈 Recettes: 220,000€
  📉 Dépenses: 9,745,392€
  ⚖️ Delta: -9,525,392€
  🏛️ Fonctions: 4
  📋 Exemples: Caen, Caen-Clos Joli Phase 1, Aeroport Caen-Carpiquet

✅ Ouistreham:
  📊 13 lignes budgétaires
  💰 Budget total: 997,263€
  📈 Recettes: 0€
  📉 Dépenses: 997,263€
  ⚖️ Delta: -997,263€
  🏛️ Fonctions: 2
  📋 Exemples: Ouistreham, Passerelle entre Ouistreham et Merville-Franceville

✅ Lion-sur-Mer:
  📊 17 lignes budgétaires
  💰 Budget total: 1,395,233€
  📈 Recettes: 211,400€
  📉 Dépenses: 1,183,833€
  ⚖️ Delta: -972,433€
  🏛️ Fonctions: 3
  📋 Exemples: Lion-sur-Mer, DMO-Lion-sur-mer-Aire de camping MO CLM, Echangeur Lion sur mer

✅ Louvigny:
  📊 9 lignes budgétaires
  💰 Budget total: 293,183€
  📈 Recettes: 0€
  📉 Dépenses: 293,183€
  ⚖️ Delta: -293,183€
  🏛️ Fonctions: 2
  📋 Exemples: Louvigny

✅ Ifs:
  📊 13 lignes budgétaires
  💰 Budget total: 3,539,245€
  📈 

In [21]:
# 📤 Création de la structure JSON finale pour le dashboard
print("📤 CRÉATION DU FICHIER JSON POUR LE DASHBOARD")
print("=" * 60)

# Mapping des fonctions pour les noms complets
fonction_mapping = {
    0: "Services généraux",
    1: "Sécurité et salubrité publiques", 
    2: "Enseignement - Formation",
    3: "Culture - Vie sociale",
    4: "Sports et jeunesse",
    5: "Interventions sociales et santé",
    6: "Famille",
    7: "Logement",
    8: "Aménagement et services urbains",
    9: "Action économique"
}

def creer_structure_dashboard(communes_data, communes_info):
    """Crée la structure JSON pour le dashboard JavaScript"""
    
    # Construire la liste des villes
    villes = []
    
    for nom_commune, analyse in communes_data.items():
        info_commune = communes_info.get(nom_commune, {})
        
        # Convertir les fonctions au format attendu
        fonctions_liste = []
        for fonction_code, fonction_data in analyse['fonctions'].items():
            fonctions_liste.append({
                'code': fonction_code,
                'nom': fonction_mapping.get(fonction_code, f"Fonction {fonction_code}"),
                'recettes': fonction_data['recettes'],
                'depenses': fonction_data['depenses'],
                'total': fonction_data['total']
            })
        
        # Structure de la ville
        ville = {
            'nom': nom_commune,
            'coordonnees': [info_commune.get('lat', 49.1829), info_commune.get('lon', -0.3707)],
            'population': info_commune.get('population', 1000),
            'budget': {
                'total': analyse['budget_total'],
                'recettes': analyse['recettes'],
                'depenses': analyse['depenses'],
                'delta': analyse['delta']
            },
            'fonctions': fonctions_liste,
            'stats': {
                'budget_par_habitant': analyse['budget_total'] / info_commune.get('population', 1),
                'nb_lignes_budget': analyse['nb_lignes']
            }
        }
        villes.append(ville)
    
    # Créer la liste des fonctions avec totaux
    toutes_fonctions = set()
    for commune_data in communes_data.values():
        toutes_fonctions.update(commune_data['fonctions'].keys())
    
    fonctions = []
    for fonction_code in sorted(toutes_fonctions):
        total_fonction = sum(
            commune['fonctions'].get(fonction_code, {}).get('total', 0)
            for commune in communes_data.values()
        )
        
        fonctions.append({
            'code': fonction_code,
            'nom': fonction_mapping.get(fonction_code, f"Fonction {fonction_code}"),
            'total': total_fonction
        })
    
    # Statistiques générales
    budget_total = sum(c['budget_total'] for c in communes_data.values())
    total_depenses = sum(c['depenses'] for c in communes_data.values())
    total_recettes = sum(c['recettes'] for c in communes_data.values())
    
    # Commune principale
    commune_principale = max(communes_data.items(), key=lambda x: x[1]['budget_total'])
    
    # Structure finale
    structure = {
        'metadata': {
            'source': 'CUCLM Budget 2020 - Analyse Optimisée',
            'date_generation': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
            'nb_communes': len(villes),
            'budget_total': budget_total,
            'methode': 'Extraction CSV avec analyse des vraies communes CUCLM'
        },
        'villes': villes,
        'fonctions': fonctions,
        'statistiques': {
            'budget_total': budget_total,
            'nb_communes': len(communes_data),
            'total_depenses': total_depenses,
            'total_recettes': total_recettes,
            'commune_principale': commune_principale[0]
        }
    }
    
    return structure

# Créer la structure JSON
print("🏗️ Construction de la structure JSON...")
dashboard_data = creer_structure_dashboard(communes_analysees, communes_cuclm)

# Afficher les statistiques
print(f"📊 STRUCTURE CRÉÉE:")
print(f"  • Communes: {len(dashboard_data['villes'])}")
print(f"  • Fonctions: {len(dashboard_data['fonctions'])}")
print(f"  • Budget total: {dashboard_data['metadata']['budget_total']:,.0f}€")

# Sauvegarder le fichier
output_file = 'budget-cuclm-optimized-final.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(dashboard_data, f, indent=2, ensure_ascii=False)

print(f"✅ Fichier sauvegardé: {output_file}")

# Afficher un échantillon
print(f"\n🔍 ÉCHANTILLON DES DONNÉES:")
if dashboard_data['villes']:
    ville_ex = dashboard_data['villes'][0]
    print(f"📍 {ville_ex['nom']}:")
    print(f"  • Coordonnées: {ville_ex['coordonnees']}")
    print(f"  • Population: {ville_ex['population']:,}")
    print(f"  • Budget: {ville_ex['budget']['total']:,.0f}€")
    print(f"  • Fonctions: {len(ville_ex['fonctions'])}")

print(f"\n🏛️ FONCTIONS BUDGÉTAIRES:")
for fonction in dashboard_data['fonctions'][:5]:
    print(f"  • {fonction['code']}: {fonction['nom']} ({fonction['total']:,.0f}€)")

print(f"\n🎯 DONNÉES PRÊTES POUR LE DASHBOARD !")
print(f"🔗 Fichier à utiliser: {output_file}")

📤 CRÉATION DU FICHIER JSON POUR LE DASHBOARD
🏗️ Construction de la structure JSON...
📊 STRUCTURE CRÉÉE:
  • Communes: 10
  • Fonctions: 7
  • Budget total: 19,102,072€
✅ Fichier sauvegardé: budget-cuclm-optimized-final.json

🔍 ÉCHANTILLON DES DONNÉES:
📍 Caen:
  • Coordonnées: [49.1829, -0.3707]
  • Population: 105,512
  • Budget: 9,965,392€
  • Fonctions: 4

🏛️ FONCTIONS BUDGÉTAIRES:
  • 0: Services généraux (666,854€)
  • 3: Culture - Vie sociale (3,060,127€)
  • 4: Sports et jeunesse (200,000€)
  • 5: Interventions sociales et santé (630,391€)
  • 6: Famille (418,574€)

🎯 DONNÉES PRÊTES POUR LE DASHBOARD !
🔗 Fichier à utiliser: budget-cuclm-optimized-final.json
